# Maximisation-expectation

Notebook configuration

In [8]:
from os import getcwd, chdir

# set the current working directory to the repository's root, if required
if getcwd().endswith('notebooks'):
    chdir('./../')

In [9]:
!python -m pip install tabulate
from IPython.display import display
import pandas as pd
pd.set_option('display.float_format', lambda x: f'{x:.4f}')

def display_horizontaly_two_tables(df:pd.DataFrame,df2:pd.DataFrame, title1:str, title2:str):
    df_md = df.to_markdown().split('\n')
    df2_md = df2.to_markdown().split('\n')
    title_line = title1 + " " * (len(df_md[0])-len(title1)) + "\t" + title2
    two_tabs = "\n".join([title_line] + [df_md[i] + '\t' + df2_md[i] for i in range(len(df_md))])
    print(two_tabs)

## Initial Dataset

In [10]:
from data.getDataset import getCognatesSet, getIteration
from data.vocab import computeInferenceData, V_SIZE
from torch.nn.utils.rnn import pad_sequence, pad_packed_sequence, pack_padded_sequence
from Source.editModel import EditModel

cognates = computeInferenceData(pad_sequence(getCognatesSet()['french'], batch_first=True))
currentReconstructions = computeInferenceData(pad_sequence(getIteration(1), batch_first=True))

randomEditModel = EditModel(cognates, 'french', 50)
x_maxLength = pad_packed_sequence(currentReconstructions[0])[0].size()[0] - 2
y_maxLength = pad_packed_sequence(cognates[0])[0].size()[0] - 2
print('|y| max =', y_maxLength)
print('|x| max =', x_maxLength)

|y| max = 18
|x| max = 16


## Backward dynamic Program

Inference in edit model

In [11]:
randomEditModel.update_cachedTargetContext()

Backward dynamic Program

In [12]:
from Source.dynamicPrograms import compute_posteriors

randomEditModel.update_cachedTargetContext()
targets_prob = compute_posteriors(randomEditModel, currentReconstructions, cognates)

### Analysis

Test on $x :=$`"absyrdʊ"` and $y :=$`"absˈyʁd"` ($|x|=7$ and $|y|=7$)

In [13]:
from torch import tensor
from numpy import array
idx = 20 # the index in the batch of the sample to be studied
x_length, y_length = 7, 7
dlt = tensor(array([[randomEditModel.dlt(i, j)[idx] for j in range(20)] for i in range(18)]))
end = tensor(array([[randomEditModel.end(i, j)[idx] for j in range(20)] for i in range(18)]))
sub = tensor(array([[randomEditModel.sub(i, j)[idx] for j in range(20)] for i in range(18)]))
ins = tensor(array([[randomEditModel.ins(i, j)[idx] for j in range(20)] for i in range(18)]))

backward_prob_dlt = targets_prob.dlt[:,:,idx]
backward_prob_end = targets_prob.end[:,:,idx]
backward_prob_sub = targets_prob.sub[:,:,idx]
backward_prob_ins = targets_prob.ins[:,:,idx]

#### Display of cached probabilities for the substitution and the deletion operations

Notice that the logarithmic probabilities that have not to be defined for these operations are automatically set to $-\infty{}$ thanks to the recurrence mathematical relations of the backward dynamic program. So do the undefined probabilities in padding positions, thanks to the neutrality of edit model's cached inference probabilities (which equal $0$ in the log space).

In [14]:
df = pd.DataFrame(backward_prob_sub[:x_length+2, :y_length+2])
df2 = pd.DataFrame(backward_prob_dlt[:x_length+2, :y_length+2])

display_horizontaly_two_tables(df, df2, "Subsitution Operation", "Deletion Operation")

Subsitution Operation                                                                                	Deletion Operation
|    |        0 |        1 |        2 |        3 |        4 |        5 |        6 |      7 |      8 |	|    |        0 |        1 |        2 |        3 |        4 |        5 |        6 |         7 |      8 |
|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|-------:|-------:|	|---:|---------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|-------:|
|  0 | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -2e+09 | -2e+09 |	|  0 | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09   | -1e+09    | -2e+09 |
|  1 | -1.29095 | -1.98294 | -2.76997 | -3.711   | -4.83866 | -6.27045 | -8.27874 | -1e+09 | -1e+09 |	|  1 | -1.50347 | -2.00356 | -2.61257 | -3.31295 | -4.15105 | -5.17892 | -6.46528 | -8.32801  | -1e+09 |
|  2 | -2.01502 | -1.88899 | -2.16685 | -2.7007  | 

## Rendering of the target and the logits before the loss computation

In [15]:
renderedTargets, renderedLogits = randomEditModel.renderTargetAndLogitsBeforeLossComputation(currentReconstructions, targets_prob)
renderedTargets = renderedTargets[:,:,idx].cpu().numpy()
renderedLogits = renderedLogits[:,:,idx].cpu().numpy()

#### Comparison of the rendered target probs format and the cached probs format

A reduction of the renderedTargets has been done for the displaying. The comparison is done for the insertion and the ending operations.

In [17]:
df_insBackward = pd.DataFrame(backward_prob_ins[:x_length+2, :y_length+2])
df_endBackward = pd.DataFrame(backward_prob_end[:x_length+2, :y_length+2])
df_insRenderedTargets = pd.DataFrame(renderedTargets[:x_length+2, :y_length+2, -V_SIZE-1:-1].sum(axis=2))
df_endRenderedTargets = pd.DataFrame(renderedTargets[:x_length+2, :y_length+2, -1])
df_insRenderedLogits = pd.DataFrame(renderedLogits[:x_length+2, :y_length+2, -V_SIZE-1:-1].sum(axis=2))
df_endRenderedLogits = pd.DataFrame(renderedLogits[:x_length+2, :y_length+2, -1])

display_horizontaly_two_tables(df_insBackward, df_endBackward, "Insertion operation - cached posterior probs", "Ending operation - cached posterior probs")
display_horizontaly_two_tables(df_insRenderedTargets, df_endRenderedTargets, "Insertion operation - rendered target probs", "Ending operation - rendered target probs")
display_horizontaly_two_tables(df_insRenderedLogits, df_endRenderedLogits, "Insertion operation - rendered logits", "Ending operation - rendered logits")

# display(pd.merge(df_insBackward, df_endBackward, left_index=True, right_index=True, suffixes=(' ins', ' end')),
#         pd.merge(df_insRenderedTargets, df_endRenderedTargets, left_index=True, right_index=True, suffixes=(' ins', ' end'))
#         )

Insertion operation - cached posterior probs                                                          	Ending operation - cached posterior probs
|    |         0 |        1 |        2 |        3 |        4 |        5 |        6 |      7 |      8 |	|    |         0 |        1 |        2 |        3 |        4 |        5 |        6 |         7 |      8 |
|---:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|-------:|-------:|	|---:|----------:|---------:|---------:|---------:|---------:|---------:|---------:|----------:|-------:|
|  0 | -0.687895 | -1.46918 | -2.36335 | -3.40391 | -4.64833 | -6.18891 | -8.32801 | -1e+09 | -1e+09 |	|  0 | -0.698427 | -1.30005 | -1.99503 | -2.79915 | -3.74374 | -4.88946 | -6.3142  | -8.32801  | -1e+09 |
|  1 | -1e+09    | -1.99044 | -2.09865 | -2.62776 | -3.47194 | -4.66584 | -6.48434 | -1e+09 | -1e+09 |	|  1 | -1e+09    | -1.97779 | -1.886   | -2.17996 | -2.72362 | -3.52123 | -4.62787 | -6.33057  | -1e+09 |
|  2 | -1e+09    | 